In [1]:
import fitsio as fio
import numpy as np
import os, sys
import glob
from matplotlib import pyplot as plt
import proplot as pplt
from tqdm import tqdm
import pickle
from des_y6utils import mdet
%matplotlib inline

os.environ["MEDS_DIR"] = "/global/cfs/cdirs/des/y6-shear-catalogs"

In [2]:
# grab mdet files (these files are divided into patches)
# mdet_files = glob.glob('/global/cfs/cdirs/des/y6-shear-catalogs/metadetection_patches_v1_blinded/patch-*.fits')
# mdet_files = glob.glob('/global/cfs/cdirs/des/y6-shear-catalogs/metadetection_Y6A2_METADETECT_V4_TEST_V1/*_metadetect.fits')
mdet_files = np.load('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/som_weight_test/som_training_files.npy')

In [3]:
# preliminary shear weight
with open('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/som_weight_test/inverse_variance_weight_som_test_20steps.pickle', 'rb') as handle:
    wgt_dict = pickle.load(handle)

In [4]:
def assign_loggrid(x, y, xmin, xmax, xsteps, ymin, ymax, ysteps):
    from math import log10
    # return x and y indices of data (x,y) on a log-spaced grid that runs from [xy]min to [xy]max in [xy]steps

    logstepx = log10(xmax/xmin)/xsteps
    logstepy = log10(ymax/ymin)/ysteps

    indexx = (np.log10(x/xmin)/logstepx).astype(int)
    indexy = (np.log10(y/ymin)/logstepy).astype(int)

    indexx = np.maximum(indexx,0)
    indexx = np.minimum(indexx, xsteps-1)
    indexy = np.maximum(indexy,0)
    indexy = np.minimum(indexy, ysteps-1)

    return indexx,indexy

def _find_shear_weight(d, wgt_dict, snmin, snmax, sizemin, sizemax, steps):
    
    if wgt_dict is None:
        weights = np.ones(len(d))
        return weights

    shear_wgt = wgt_dict['weight']
    shear_response = wgt_dict['response']
    indexx, indexy = assign_loggrid(d['wmom_s2n'], d['wmom_T_ratio'], snmin, snmax, steps, sizemin, sizemax, steps)
    weights = np.array([shear_wgt[x, y] for x, y in zip(indexx, indexy)])
    response = np.array([shear_response[x, y] for x, y in zip(indexx, indexy)])
    
    return response, weights

In [5]:
f_response = open('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/som_weight_test/weighted_shear_response_som_weight.txt', 'r')
R11, R22 = f_response.read().split('\n')
R = (float(R11) + float(R22))/2

In [6]:
R

0.2598280498420682

In [7]:
# Put all the objects in an array.
obj_list = []
for i,fname in tqdm(enumerate(mdet_files)):
    
    # try:
    d = fio.read(fname)
    # msk = mdet.make_mdet_cuts(d, 3) 
    msk = mdet._make_mdet_cuts_wmom(
            d,
            min_s2n=5.0,
            min_t_ratio=1.1,
            n_terr=0.0,
            max_mfrac=0.1,
            max_s2n=np.inf,
        )
    noshear_mask = (d['mdet_step'] == 'noshear')
    d = d[msk & noshear_mask]
    # except:
    #     print('error cutting')
    #     continue
    
    # mdet_obj = np.zeros(len(d), dtype=[('tilename', object), ('slice_id', 'i2'), ('x', 'f8'), ('y', 'f8'), ('ra', 'f8'), ('dec', 'f8')]) #, ('s2n', 'f8'), ('g1', 'f8'), ('g2', 'f8'), ('g1_cov', 'f8'), ('g2_cov', 'f8')])
    mdet_obj = np.zeros(len(d), dtype=[('ra', 'f8'), ('dec', 'f8'), ('s2n', 'f8'), ('g1', 'f8'), ('g2', 'f8'), ('g1_cov', 'f8'), ('g2_cov', 'f8'), ('R_all', 'f8'), ('w', 'f8')])
    shear_response, shear_wgt = _find_shear_weight(d, wgt_dict, 10, 600, 1.2, 2.0, 20)
    # shear_wgt = 1/(0.07**2 + 0.5*(d['wmom_g_cov_1_1'] + d['wmom_g_cov_2_2']))
    
    mdet_mom = 'wmom'
    # mdet_obj['tilename'] = d['tilename']
    # mdet_obj['slice_id'] = d['slice_id']
    # mdet_obj['x'] = d['x']
    # mdet_obj['y'] = d['y']
    mdet_obj['ra'] = d['ra']
    mdet_obj['dec'] = d['dec']
    mdet_obj['s2n'] = d[mdet_mom+'_s2n']
    mdet_obj['g1'] = d[mdet_mom+'_g_1']
    mdet_obj['g2'] = d[mdet_mom+'_g_2']
    mdet_obj['g1_cov'] = d[mdet_mom+'_g_cov_1_1']
    mdet_obj['g2_cov'] = d[mdet_mom+'_g_cov_2_2']
    mdet_obj['R_all'] = R
    mdet_obj['w'] = shear_wgt
    
    # obj_list.append(mdet_obj)
    if i == 0:
        mdet_all = mdet_obj
    else:
        mdet_all = np.concatenate([mdet_all, mdet_obj])
fio.write('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/som_weight_test/metadetection_som_inverse_weight_20steps.fits', mdet_all)

50it [19:05, 22.91s/it]


In [ ]:
fio.write('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/y6_catalog4matching.fits', mdet_all)

# Analysis Choices

In [ ]:
d = fio.read('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/mean_shear_stats_final_v1.fits')

In [ ]:
fig,ax=plt.subplots(nrows=1, ncols=2, figsize=(7,3))
ax[0].hist(np.log10(d['wmom_s2n']), bins=500, histtype='step')
ax[1].hist(d['wmom_T_ratio'], bins=500, histtype='step')
plt.show()

In [ ]:
print('S/N cut 300', np.mean((d['wmom_s2n'] < 300)))
print('S/N cut 500', np.mean((d['wmom_s2n'] < 500)))

In [ ]:
print('T_ratio cut 2.0', np.mean((d['wmom_T_ratio'] < 2.0)))

In [ ]:
d_v2_snweight = fio.read('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/metadetection_v2_snweight.fits')

In [ ]:
fig,ax=plt.subplots(figsize=(4,3))
h = ax.hist(d_v2_snweight['w'], histtype='step', bins=500)
wgt2 = 1/(0.07**2 + )
h = ax.hist(wgt2, histtype='step', bins=500)
ax.set_xlabel('per-galaxy weight')
plt.tight_layout()

In [ ]:
fig,ax=plt.subplots(figsize=(4,3))
wgt = 1/(0.25**2 + 0.5*(d_v2_snweight['g1_cov'] + d_v2_snweight['g2_cov']))
h = ax.hist(wgt, histtype='step', bins=500)
ax.set_xlabel('per-galaxy weight')
plt.tight_layout()